In [1]:
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split
import pickle

In [2]:
!pip3 install --upgrade alexnet_pytorch

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from alexnet_pytorch import AlexNet
import torch.optim as optim

In [4]:
rdir_path = './newData'

In [6]:
imgs = []
labels = []
label = -1
onlyDirs = [f for f in os.listdir(rdir_path) if os.path.isdir(os.path.join(rdir_path, f))]
for directory in onlyDirs:
    label = label + 1
    path = rdir_path+'/'+directory
    onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    for file in onlyfiles:
        imgPath = path+'/'+file
        imgs.append(imgPath)
        labels.append(label)

In [7]:
print(len(imgs))

179


In [8]:
trainImgPaths,testImgPaths, trainLabels, testLabels = train_test_split(imgs, labels, 
                                                train_size=0.8, random_state=42,shuffle = True,stratify = labels)

In [12]:
train_imgs = []
for imgName in trainImgPaths:
    img = cv2.imread(imgName)
    train_imgs.append(img)
train_imgs = np.asarray(train_imgs)

In [13]:
testImgs = []
for imgName in testImgPaths:
    img = cv2.imread(imgName)
    testImgs.append(img)
testImgs = np.asarray(testImgs)

In [9]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /home/amithareddy/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/amithareddy/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [10]:
AlexNet_model.classifier[4] = nn.Linear(4096,1024)
AlexNet_model.classifier[6] = nn.Linear(1024,2)

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.001,momentum=0.9)

In [21]:
print(train_imgs.size)

21235500


In [24]:
import tensorflow as tf
import numpy as np

2022-03-30 20:11:38.017797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/amithareddy/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-30 20:11:38.017905: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [25]:
tensor_train_imgs = tf.convert_to_tensor(train_imgs)
tensor_trainLabels = tf.convert_to_tensor(trainLabels)

2022-03-30 20:12:07.645034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/amithareddy/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-30 20:12:07.645170: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-30 20:12:07.645242: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (amithareddy-Inspiron-5570): /proc/driver/nvidia/version does not exist
2022-03-30 20:12:07.684206: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-3

In [26]:
start_time = time.time()
for iteration in range(10):  # loop over the dataset multiple times

    iteration_loss = 0.0
    for i, inputs, labels in zip( range(train_imgs.size), train_imgs, trainLabels) :

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = AlexNet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        ieration_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (iteration + 1, i + 1, iteration_loss / 1000))
            # print(loss.item())
            iteration_loss = 0.0

print('Training Completed')
print("Time taken to train AlexNet is %s seconds" % (time.time() - start_time))

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [253]:
descriptors = getdescriptors(train_imgs)
kmeans = KMeans(n_clusters = nClusters)
kmeans.fit(descriptors)

KMeans(n_clusters=80)

In [254]:
train_features = getfeatures(trainImgs,kmeans)

In [255]:
model = svm.SVC(C=0.1,kernel='linear')
model.fit(train_features,trainLabels)

SVC(C=0.1, kernel='linear')

In [247]:
testImgs = []
for imgName in testImgPaths:
    img = cv2.imread(imgName)
    testImgs.append(img)
testImgs = np.asarray(testImgs)
test_features = getfeatures(testImgs,kmeans)

In [248]:
acc = model.score(test_features,testLabels)
print(acc)

0.9444444444444444


In [249]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(train_features,trainLabels)

KNeighborsClassifier(n_neighbors=10)

In [250]:
acck = neigh.score(test_features,testLabels)
print(acck)

0.9166666666666666


In [251]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter = 5000).fit(train_features,trainLabels)

In [252]:
acclr = clf.score(test_features,testLabels)
print(acclr)

0.9444444444444444
